get only no of classes /2 

In [5]:
import pandas as pd
import json
import os

results_csv_path = '/microDec/data/final_results/final_results.csv'
metrics_dir = '/microDec/data/all_results_with_scoh_OpenAI/'

try:
    results_df = pd.read_csv(results_csv_path)
except FileNotFoundError:
    print(f" {results_csv_path} not exist")
    results_df = pd.DataFrame()

def get_best_clustering_results(json_path, max_clusters):
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    def calculate_score(metrics):
        w_chm = 1.0
        w_chd = 1.0
        w_smq = 1.0
        w_cmq = 1.0
        w_ifn = -1.0
        
        score = (w_chm * metrics['CHM'] + 
                 w_chd * metrics['CHD'] + 
                 w_smq * metrics['SMQ'] + 
                 w_cmq * metrics['CMQ'] + 
                 w_ifn * metrics['IFN'])
        return score
    
    best_score = float('-inf')
    best_method = None
    best_metrics = None
    best_clusters = None

    for method, details in data.items():
        # Extract the number of clusters from the method name, assuming format "KMeans_X"
        num_clusters = int(method.split('_')[1])
        
        if num_clusters <= max_clusters:
            metrics = details['metrics']
            score = calculate_score(metrics)
            
            if score > best_score:
                best_score = score
                best_method = method
                best_metrics = metrics
                best_clusters = details['named_clusters']
    
    return best_method, best_metrics, best_clusters

# Process each row
if not results_df.empty:
    for idx, row in results_df.iterrows():
        repo_name = row['repo']
        num_classes = row['classes']
        max_clusters = num_classes // 2
        
        json_filename = f"metrics_results_temp_{repo_name}.json"
        json_path = os.path.join(metrics_dir, json_filename)
        
        if os.path.exists(json_path):
            best_method, best_metrics, best_clusters = get_best_clustering_results(json_path, max_clusters)
            
            if best_method:
                results_df.at[idx, 'Clustering_Method'] = best_method
                results_df.at[idx, 'CHM'] = best_metrics['CHM']
                results_df.at[idx, 'CHD'] = best_metrics['CHD']
                results_df.at[idx, 'IFN'] = best_metrics['IFN']
                results_df.at[idx, 'CMQ'] = best_metrics['CMQ']
                results_df.at[idx, 'ccoh'] = best_metrics['ccoh']
                results_df.at[idx, 'ccop'] = best_metrics['ccop']
                results_df.at[idx, 'SMQ'] = best_metrics['SMQ']
                results_df.at[idx, 'scoh'] = best_metrics['scoh']
                results_df.at[idx, 'scop'] = best_metrics['scop']
                results_df.at[idx, 'SERVICES'] = best_method
                #results_df.at[idx, 'clustering_names'] = str(best_clusters)
        else:
            print(f" {json_filename} not exist")

    results_df.to_csv(results_csv_path, index=False)
else:
    print("resultsis empty")


In [ ]:
import json


with open('/microDec/metrics_results_production_ssm-master_TEST.json', 'r') as file:
    data = json.load(file)




def calculate_score(metrics):
    
    w_chm = 1.0
    w_chd = 1.0
    w_smq = 1.0
    w_cmq = 1.0
    w_ifn = -1.0
    


    score = (w_chm * metrics['CHM'] + 
             w_chd * metrics['CHD'] + 
             w_smq * metrics['SMQ'] + 
             w_cmq * metrics['CMQ'] + 
             w_ifn * metrics['IFN'])
    return score




best_score = float('-inf')
best_method = None
best_metrics = None
best_clusters = None

for method, details in data.items():
    metrics = details['metrics']
    score = calculate_score(metrics)
    
    if score > best_score:
        best_score = score
        best_method = method
        best_metrics = metrics
        best_clusters = details['named_clusters']


print(f"Best: {best_method}")
print(f"score: {best_score}")
print(f"metrics: {best_metrics}")
 
